In [21]:
from utils import *
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.decomposition import PCA

In [22]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")

In [23]:
selected_feature_model = "fc_fd"

t = int(input("Enter value of t: "))
if t < 1:
    raise ValueError("t should be a positive integer")

num_layers = int(input("Enter the number of layers: "))
if num_layers < 1:
    raise ValueError("num_layers should be a positive integer")

num_hashes_per_layer = int(input("Enter the number of hashes per layer: "))
if num_hashes_per_layer < 1:
    raise ValueError("num_hashes_per_layer should be a positive integer")

In [24]:
all_images = list(fd_collection.find())
all_images = sorted(all_images, key = lambda x: x["image_id"])

odd_image_ids = [img["image_id"] for img in all_images if img["image_id"] % 2 == 0]

even_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 == 0]
odd_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 != 0]

feature_vectors = [np.array(img[selected_feature_model]).flatten() for img in all_images]

total_len = len(feature_vectors)
even_feature_vectors = []
odd_feature_vectors = []

for i in range(total_len):
  if i % 2 == 0:
    even_feature_vectors.append(feature_vectors[i])
  else:
    odd_feature_vectors.append(feature_vectors[i])


even_feature_vectors = np.array(even_feature_vectors)
odd_feature_vectors = np.array(odd_feature_vectors)

odd_len = odd_feature_vectors.shape[0]
even_len = even_feature_vectors.shape[0]

In [25]:
num_vectors = even_feature_vectors.shape[0]
vector_dimension = even_feature_vectors.shape[1]

# lsh = LSHIndex(num_layers, num_hashes_per_layer, vector_dimension)

# for index, vector in enumerate(even_feature_vectors):
#     lsh.add_vector(vector.tolist(), index * 2)

lsh = LSH(even_feature_vectors, num_layers, num_hashes_per_layer)

In [28]:
query_image_id = int(input("Enter value of query_image_id: "))
if query_image_id < 1:
    raise ValueError("query_image_id should be a positive integer")

query_vector = np.array(odd_feature_vectors[(query_image_id // 2)])

similar_images = lsh.find_similar(query_vector, t)
unique_images_count = lsh.get_unique_images_considered_count()
overall_images_count = lsh.get_overall_images_considered_count()


print("Number of unique images considered:", unique_images_count)
print("Overall number of images considered:", overall_images_count, "\n")

print(f"Input image ID: {query_image_id}")
print(f"Input image label: {odd_image_labels[query_image_id // 2]}")
print(f"{len(similar_images)} images")
for sim in similar_images:
    print(f"Label: {even_image_labels[sim]}, ID: {sim * 2}")

Number of unique images considered: 1842
Overall number of images considered: 1842 

Input image ID: 8675
Input image label: 100
10 images
Label: 61, ID: 6328
Label: 80, ID: 7314
Label: 100, ID: 8630
Label: 80, ID: 7306
Label: 99, ID: 8600
Label: 63, ID: 6482
Label: 100, ID: 8624
Label: 99, ID: 8594
Label: 100, ID: 8626
Label: 6, ID: 2724
